# Lektion 11 - Prestandaoptimering och fine-tuning

**Assignment: Transfer learning and data pipeline tuning**

Instructions:
1. Use a pretrained model (e.g., ResNet18)
2. Compare frozen vs fine-tuned performance
3. Measure small performance tweaks

## Task 1: Transfer learning
Start with a pretrained model and a new classifier head.

In [28]:
# TODO: Load a pretrained model
import torch
from torchvision import models

# Vi hämtar hem resnet mha torchsvision, och hämtar vikterna från varianten
# som är tränad på ImageNets
model = models.resnet18(weights = models.ResNet18_Weights.IMAGENET1K_V1)

In [29]:
# Vi tar en span på modellen

model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [30]:
# TODO: Freeze the base layers
from torch.nn import Linear

num_feats = model.fc.in_features
model.fc = Linear(num_feats, 10)

# Här sätter vi gradient till false, alltså
# vikterna kommer inte uppdateras vid backprop
# Vi börjar med att freeza alla lager
for param in model.parameters():
    param.requires_grad = False

# Sedan tinar vi vårt fully connected layer på slutet
for param in model.fc.parameters():
    param.requires_grad = True



In [31]:
# Move the model to a gpu device

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
# Vi behöver ett dataset att träna modellen på! 
# Vi kör CIFAR-10

from torchvision import datasets
# The downloaded data is in the form of PIL images, we need to transform them to tensors
from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor()
])


train_data = datasets.CIFAR10(root = "data", train = True, download = True, transform = transform)
test_data = datasets.CIFAR10(root = "data", train = False, download = True, transform = transform)


# Vi behöver också en dataloader
from torch.utils.data import DataLoader

# Shuffle = True för träning, False för testning för att bibehålla ordningen på testdata
train_dataloader = DataLoader(train_data, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 32, shuffle = False)   

Files already downloaded and verified
Files already downloaded and verified


In [33]:
# TODO: Train a new classifier head
import torch.optim as optim
import torch.nn as nn

# Istället för att stoppa in model.parameters() 
# Så stoppar vi nu bara in model.fc.parameters()
# Alltså parametrarna i vårt Fully connected layer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Träningsloop
num_epochs = 1
for _ in range(num_epochs):
    model.train()
    for xb, yb in train_dataloader:
        # Vi ser till att flytta vår batchdata till devicen!
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()



In [42]:
# TODO: Record accuracy

def eval_acc(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = torch.argmax(model(xb), dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    return correct / max(total, 1)

train_acc = eval_acc(model, train_dataloader)
test_acc = eval_acc(model, test_dataloader)

print(f"Train Accuracy: {train_acc:.2f}")
print(f"Test Accuracy: {test_acc:.2f}")

Train Accuracy: 0.45
Test Accuracy: 0.44


## Task 2: Fine-tuning
Unfreeze part of the base and compare performance.

In [44]:
# TODO: Unfreeze part of the base

# Nu, så unfreezar vi en del av basen, t.ex layer4
for name, param in model.named_parameters():
    if name.startswith("layer4"):
        param.requires_grad = True

# I uttrycket nedan gör vi lite pythonhax för att sätta alla lager
# där parametrarna.requires_grad = True, och ger de en superlåg learning rate
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005)




In [45]:
# TODO: Train again and record accuracy

num_epochs = 1
for _ in range(num_epochs):
    model.train()
    for xb, yb in train_dataloader:
        # Vi ser till att flytta vår batchdata till devicen!
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
acc_tuned = eval_acc(model, test_dataloader)
print(f"Fine-tuned accuracy: {acc_tuned:.4f}")

Fine-tuned accuracy: 0.6720


In [46]:
# TODO: Compare with Task 1

# TODO: Record accuracy

def eval_acc(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = torch.argmax(model(xb), dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    return correct / max(total, 1)

train_acc = eval_acc(model, train_dataloader)
test_acc = eval_acc(model, test_dataloader)

print(f"Train Accuracy: {train_acc:.2f}")
print(f"Test Accuracy: {test_acc:.2f}")

Train Accuracy: 0.73
Test Accuracy: 0.67


## Task 3: Dataloader tuning
Measure the effect of data loader settings.

In [ ]:
# FUN FACTS!

#  TODO: Test at least two of:
# - num_workers (bra på linux, ok på windows, kass på mac)
# - pin_memory (bra på linux och windows, kass på mac)
# - batch size (gör så stor som möjligt, börja på 32)


# Olika datorer har olika hårdvara och mjukvara
# Vi vill i regel distribuera datorkraft så optimalt som möjligt
# För vår ML-modell. 

# num_workers är ett sätt att dela upp hur många enheter som gör beräkning samtidigt
# MEN: på Windows och Mac är instantieriengen av Spawn-typ (den skapar en helt ny process, tar lång tid)
# På linux är det Fork-typ, en direkt kopia, mer eller mindre direkt
# Det är värt att hålla koll på iaf. 

# Batch size är hur mycket data som matas in i modellen åt gången. 
# Batch size är också något som kan optimera tidseffektivitet och minneshantering under träning
# Vi vill typiskt ha så stor batch som möjligt
# MEN: för stor batch size kan göra att datorn får slut på minne
# Så vi börjar typiskt med 32, och kan sedan stega upp i 2-potenser (32, 64, 128, 256, 512, ...)

# Olika typer av datorer har olika minneshantering. 
# På Mac är det Unified RAM, 
# På windows Fixed VRAM
# På linux Fixed

In [39]:
# TODO: Record training time for 1-2 epochs

In [40]:
print("Done! You explored fine-tuning and performance tuning.")

Done! You explored fine-tuning and performance tuning.
